In [2]:
print(df.columns.tolist())


['mol_idx', 'atom_index', 'element_x', 'shift_ppm_x', 'element_y', 'shift_ppm_y', 'hose_1', 'hose_2', 'hose_3', 'hose_4', 'bit_0', 'bit_1', 'bit_2', 'bit_3', 'bit_4', 'bit_5', 'bit_6', 'bit_7', 'bit_8', 'bit_9', 'bit_10', 'bit_11', 'bit_12', 'bit_13', 'bit_14', 'bit_15', 'bit_16', 'bit_17', 'bit_18', 'bit_19', 'bit_20', 'bit_21', 'bit_22', 'bit_23', 'bit_24', 'bit_25', 'bit_26', 'bit_27', 'bit_28', 'bit_29', 'bit_30', 'bit_31', 'bit_32', 'bit_33', 'bit_34', 'bit_35', 'bit_36', 'bit_37', 'bit_38', 'bit_39', 'bit_40', 'bit_41', 'bit_42', 'bit_43', 'bit_44', 'bit_45', 'bit_46', 'bit_47', 'bit_48', 'bit_49', 'bit_50', 'bit_51', 'bit_52', 'bit_53', 'bit_54', 'bit_55', 'bit_56', 'bit_57', 'bit_58', 'bit_59', 'bit_60', 'bit_61', 'bit_62', 'bit_63', 'bit_64', 'bit_65', 'bit_66', 'bit_67', 'bit_68', 'bit_69', 'bit_70', 'bit_71', 'bit_72', 'bit_73', 'bit_74', 'bit_75', 'bit_76', 'bit_77', 'bit_78', 'bit_79', 'bit_80', 'bit_81', 'bit_82', 'bit_83', 'bit_84', 'bit_85', 'bit_86', 'bit_87', 'bit_88'

In [3]:
import shutil, pathlib
src = pathlib.Path(r"C:\DOCTORAL HUB\nmr_pipeline_project\data\merged\merged_phase3_clean.csv")
dst = src.with_name("merged_phase3_clean_BACKUP.csv")
shutil.copy(src, dst)
print(" Backup created:", dst)


 Backup created: C:\DOCTORAL HUB\nmr_pipeline_project\data\merged\merged_phase3_clean_BACKUP.csv


In [5]:
for nuc in sorted(df["element_y"].dropna().unique()):
    sub = df[df["element_y"] == nuc].copy()


In [6]:
ban_cols = [
    "mol_idx", "atom_index",
    "element_x", "element_y",
    "shift_ppm_x", "shift_ppm_y"
]


In [7]:
TARGET_COL = "shift_ppm_y"   #  true experimental shift


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
from math import sqrt

results = []

# Loop through each unique element
for nuc in sorted(df["element_y"].dropna().unique()):
    print(f"\n==========================")
    print(f"Training element = {nuc}")

    # Filter data for this element
    sub = df[df["element_y"] == nuc].copy()
    if len(sub) < 100:
        print(f"Skipping {nuc} (too few samples)")
        continue

    # Select features
    feature_cols = [c for c in sub.columns if c not in ban_cols and np.issubdtype(sub[c].dtype, np.number)]
    X = sub[feature_cols].values
    y = sub[TARGET_COL].values

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Ridge Regression
    ridge = Ridge(alpha=1.0, random_state=42)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    results.append({"Element": nuc, "Model": "Ridge", "R2": r2, "RMSE": rmse, "MAE": mae})
    print(f"[{nuc}] Ridge → R²={r2:.3f} | RMSE={rmse:.3f} | MAE={mae:.3f}")

    # Random Forest
    rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    results.append({"Element": nuc, "Model": "RandomForest", "R2": r2, "RMSE": rmse, "MAE": mae})
    print(f"[{nuc}] RandomForest → R²={r2:.3f} | RMSE={rmse:.3f} | MAE={mae:.3f}")

# Save results
df_results = pd.DataFrame(results)
df_results.to_csv("results/phase4b_per_element_results.csv", index=False)
print("\n Results saved to results/phase4b_per_element_results.csv")
df_results



Training element = B
[B] Ridge → R²=0.126 | RMSE=23.113 | MAE=18.811
[B] RandomForest → R²=0.088 | RMSE=23.608 | MAE=18.739

Training element = C
[C] Ridge → R²=0.085 | RMSE=49.486 | MAE=42.445
[C] RandomForest → R²=0.093 | RMSE=49.285 | MAE=42.021

Training element = Cl
Skipping Cl (too few samples)

Training element = F
[F] Ridge → R²=0.117 | RMSE=73.139 | MAE=53.520
[F] RandomForest → R²=0.159 | RMSE=71.365 | MAE=50.265

Training element = Ge
Skipping Ge (too few samples)

Training element = H
[H] Ridge → R²=0.330 | RMSE=26.586 | MAE=13.441
[H] RandomForest → R²=0.510 | RMSE=22.738 | MAE=9.136

Training element = K
Skipping K (too few samples)

Training element = N
[N] Ridge → R²=0.053 | RMSE=60.469 | MAE=46.120
[N] RandomForest → R²=-0.165 | RMSE=67.057 | MAE=50.210

Training element = O
[O] Ridge → R²=0.164 | RMSE=96.698 | MAE=83.593
[O] RandomForest → R²=0.178 | RMSE=95.862 | MAE=81.194

Training element = P
[P] Ridge → R²=0.231 | RMSE=43.632 | MAE=29.862
[P] RandomForest → R²=0

,Element,Model,R2,RMSE,MAE
0,B,Ridge,0.125549,23.113217,18.811154
1,B,RandomForest,0.087722,23.607847,18.738606
2,C,Ridge,0.085222,49.486460,42.444718
3,C,RandomForest,0.092665,49.284720,42.021235
4,F,Ridge,0.116839,73.139282,53.520058
5,F,RandomForest,0.159166,71.365125,50.265416
6,H,Ridge,0.329690,26.586417,13.441010
7,H,RandomForest,0.509715,22.737688,9.135926
8,N,Ridge,0.052926,60.468783,46.119888
9,N,RandomForest,-0.164681,67.056785,50.209993
